In [1]:
from pathlib import Path
import itertools
import multiprocessing as mp

import numpy as np
import pandas as pd

import kmers

In [2]:
kmers.seq_to_base4("ATCG")

'0312'

In [4]:
def detect_kmers_across_sequences_mp(sequences: list,
                                     kmer_size: int = 8,
                                     num_processes: int = 4,
                                     verbose: bool = False) -> list:
    if verbose:
        print("Detecting kmers across sequences mp")
    with mp.Pool(num_processes) as pool:
        args_list = [(seq, kmer_size) for seq in sequences]
        collected_results = pool.starmap_async(kmers.detect_kmers, args_list)
        results = collected_results.get()
    return results

In [3]:
ref_data = Path("data/trainset19_072023_db.csv")

ref_db = pd.read_csv(ref_data)

ref_db_sample = ref_db.sample(1000)

In [5]:
def genus_conditional_prob(detect_list: list[list],
                           genera_idx: list,
                           kmer_size: int = 8):
    kmers_idx_array = np.array(detect_list, dtype=object)
    genus_counts = np.zeros((len(detect_list), 4**kmer_size), dtype=int)
    row_idx = np.arange(len(kmers_idx_array)).repeat(np.vectorize(len)(kmers_idx_array))
    col_idx = np.concatenate(kmers_idx_array)

    genus_counts[row_idx, col_idx] = 1

    genus_count = np.append(genera_idx, genus_counts, axis=1)
    genera_idx = genus_count[:, 0].astype(int)
    genus_count_data = genus_count[:, 1:]
    unique_genera, inverse_indices = np.unique(genera_idx, return_inverse=True)
    genus_count_filled = np.zeros((len(unique_genera), genus_count_data.shape[1]))
    np.add.at(genus_count_filled, inverse_indices, genus_count_data)

    return genus_count_filled

kmr_size = 3
#
# kmers_idx_list = [
#     [0, 10, 12, 4, 50],
#     [12, 21, 3, 60, 10, 5],
#     [1, 3, 21, 55]
# ]
#
# kmers_idx_array = np.array(kmers_idx_list, dtype=object)
#
# kmers_arr = np.zeros((len(kmers_list), 4**kmr_size))
#
# row_idx = np.arange(len(kmers_idx_array)).repeat(np.vectorize(len)(kmers_idx_array))
#
# col_indices = np.concatenate(kmers_idx_array)
#
# # Use advanced indexing to populate the array
# kmers_arr[row_idx, col_indices] = 1

detect_list = detect_kmers_across_sequences_mp(ref_db_sample["sequences"], kmer_size=kmr_size)

genera_idx = np.array(kmers.genera_str_to_index(ref_db_sample["taxonomy"]), dtype=int)
genera_idx = genera_idx.reshape(-1,1)

genus_counts = genus_conditional_prob(detect_list, genera_idx, kmer_size=kmr_size)


In [ ]:



detect_arr = genus_conditional_prob(detect_list, kmer_size=kmr_size)
print("detect_arr complete")

new_arr = np.append(genera_idx, detect_arr, 1)

genera_idx = new_arr[:, 0].astype(int)
data = new_arr[:, 1:]

unique_genera, inverse_indices = np.unique(genera_idx, return_inverse=True)
result = np.zeros((len(unique_genera), data.shape[1]))
np.add.at(result, inverse_indices, data)

print("done with genus counts")
# If you need a DataFrame as the final output
# df = pd.DataFrame(result, index=unique_genera)

# df.shape

In [ ]:
my_arr = np.array(([1,1,2,2,3,3,4,4,4,2,2,1,]))
un, inv = np.unique(my_arr, return_inverse=True)
print(un)
p